# 自動機械学習 Automated Machine Learning による自動車価格予測モデリング

中古車の属性データから価格を予測するモデルを構築します。
- Python SDK のインポート
- Azure ML service Workspace への接続
- Experiment の作成
- データの準備
- 自動機械学習の事前設定
- モデル学習と結果の確認

## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning service の Python SDKをインポートします

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

W0919 00:15:35.249004 4391175616 deprecation_wrapper.py:119] From /Users/konabuta/miniconda3/envs/myenv/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0919 00:15:35.250658 4391175616 deprecation_wrapper.py:119] From /Users/konabuta/miniconda3/envs/myenv/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



### Azure ML workspace との接続
Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

azureml	eastus	mlservice	eastus


### 実験名の設定

In [3]:
# choose a name for experiment
experiment_name = 'automl-regression-automobile-simple'
experiment=Experiment(ws, experiment_name)

### 学習データの準備

In [4]:
df = Dataset.get(ws, name='automobile').to_pandas_dataframe()

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["price"],axis=1)
y = df["price"].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=100)

## 2. 自動機械学習 Automated Machine Learning
### 学習事前設定

In [6]:
automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations": 5,
    "n_cross_validations": 2,
    "primary_metric": 'normalized_mean_absolute_error',
    "preprocess": True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'regression',
                             X = X_train,
                             y = y_train,
                             **automl_settings
                            )

### 実行と結果確認

In [7]:
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_0d5bc62c-af7d-4186-b69e-d3ed5fe1b3e7
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       FIXED
DESCRIPTION:  The training data had the following missing values which were resolved.

Please review your data source for data quality issues and possibly filter out the rows with these missing values.

If the missing values are expected, you can either accept the above imputation, or implement

In [8]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
best_run, fitted_model = local_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-regression-automobile-simple,AutoML_0d5bc62c-af7d-4186-b69e-d3ed5fe1b3e7_4,,Completed,Link to Azure Portal,Link to Documentation
